# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto (Week 3)

##### Welcome back to the 2nd part of this assignment. In this assignment we need to get the latitude and the longitude coordinates of each neighborhood

##### Let's quickly get the dataframe ready from Part 1

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find("table", { "class" : "wikitable sortable" })

In [2]:
table = soup.find("table", { "class" : "wikitable sortable" })

In [3]:
Postcode=[]
Borough=[]
Neighbourhood=[]

for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True).replace("\n", ""))

In [4]:
df=pd.DataFrame(Postcode,columns=['Postalcode'])
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood

df["Borough"].replace("Not assigned", np.nan, inplace=True)

df.dropna(subset=["Borough"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df = df.groupby(['Postalcode','Borough'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

In [6]:
for i in df.index:
    if df.at[i, 'Neighbourhood'] == "Not assigned":
        print("Replaced for ", df.at[i, 'Neighbourhood'],  "with", df.at[i, 'Borough'], "at index ", i )
        df.at[i, 'Neighbourhood'] = df.at[i, 'Borough']
        


Replaced for  Not assigned with Queen's Park at index  85


In [7]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Great, now let's get the coordinates from the csv file provided in to a new dataframe

In [8]:
filename = "https://cocl.us/Geospatial_data"
headers = ["PostalCode","Latitude","Longitude"]

df2 = pd.read_csv(filename, names = headers)

In [9]:
df2.head()

,PostalCode,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


##### Add new columns to our original/main dataframe

In [10]:
df['Latitude'] = ''
df['Longitude'] = ''

##### Now merge the 2nd dataframe's Latitude and Longitude columns to the main dataframe matching the Postalcode column

In [11]:
df.loc[df.Postalcode.isin(df2.PostalCode), ['Latitude', 'Longitude']] = df2[['Latitude', 'Longitude']]


##### Here we go, we now have the Latitude and Longitude columns merged to our main dataframe. Let's have a final look at the entire dataframe. Thanks, again, for reviewing this notebook. Catch you at the final section of this week's assignment. Take care!

In [12]:
df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",Latitude,Longitude
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.8066863,-79.1943534
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7845351,-79.1604971
3,M1G,Scarborough,Woburn,43.7635726,-79.1887115
4,M1H,Scarborough,Cedarbrae,43.7709921,-79.2169174
5,M1J,Scarborough,Scarborough Village,43.773136,-79.2394761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7447342,-79.2394761
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7279292,-79.2620294
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7111117,-79.2845772
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.716316,-79.2394761
